# Veritas Fairness Assement - Credit Scoring Case Study (sample code)
This notebook includes samples of code used in the analysis conducted during the credit scoring case study.

It is applicable to a variety of publically available credit scoring datasets.
Here it is run on the 
[Kaggle hosted UCI credit card dataset](https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset) [1,2]. 

1. Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. 
Irvine, CA: University of California, School of Information and Computer Science.

2. Yeh, I. C., & Lien, C. H. (2009). The comparisons of data mining techniques for the predictive accuracy of probability of default of credit card clients. Expert Systems with Applications, 36(2), 2473-2480. 



## License
Written by Marc-Etienne Brunet,
Element AI inc. (info@elementai.com).

Copyright © 2020 Monetary Authority of Singapore

Licensed under the Apache License, Version 2.0 (the "License"); you may not use
this file except in compliance with the License. You may obtain a copy of the
License at http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the License.

## Imports

In [1]:
# Core Packages
import os
import json

# Third Party
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_score
from sklearn.calibration import calibration_curve
from sklearn import metrics

In [4]:
# Our code (autoreload)
%load_ext autoreload
%autoreload 2
import utils.credit as utils

In [5]:
# High-res plots
%config InlineBackend.figure_format = 'retina'

## AIDA System Objective

The objective of a credit approval system is to provide loans in a way that is advantageous for both the customer and for the bank.
A “good” loan is one that gets repaid. It generates revenue for the bank. A “bad” loan is one ending in default, it incurs costs for the bank. Typically, the cost from a bad loan is much higher than the revenue from a good loan.
Without some kind of lift from a predictive model, pricing must be set using the expected base default rate. 
To be more competitive, profitable, and to manage risk, banks use credit scoring models to estimate an applicant’s creditworthiness (or risk of default). 


In [6]:
def gross_profit_rate(approval_rate, default_rate, revenue=10, cost=100):
    return approval_rate * (revenue * (1 - default_rate) - cost * default_rate)

approval_rates = np.linspace(0, 1, 100)
default_rates = np.linspace(0, 0.2, 100)
grid_approval, grid_defaults = np.meshgrid(approval_rates, default_rates, sparse=True)
profit_grid = gross_profit_rate(grid_approval, grid_defaults)

In [5]:
# Visualize
plt.figure(figsize=(7, 6))
profit_lns = plt.contourf(approval_rates, default_rates, profit_grid, cmap='PiYG', levels=50)
plt.title('Credit Approval', fontsize=18)
plt.xlabel('loan approval rate', fontsize=14)
plt.ylabel('default rate', fontsize=14)
plt.axhline(0.08, c='black', ls=':', label='base default rate')
x_eg = np.linspace(0, 1, 100)
y_eg = 0.04 * np.sin(np.pi *(x_eg - 0.5)) + 0.04
plt.plot(x_eg, y_eg, label='hypothetical model', ls='--', c='grey')
cbar = plt.colorbar(profit_lns)
cbar.set_label('gross profit rate ($ / applicant)', fontsize=14)
plt.legend(fontsize=12)
plt.show()

For simplicity, we will use balanced accuracy (the average of true positive and true negative rates) as a proxy for gross profit. 
We show why a (possibly weighted) balanced accuracy is a reasonable proxy for gross profit in the report. 

## Load Data

In [6]:
# Extraneous columns (as might be determined through feature selection)
drop_cols = ['ID']

# Load into Dataframes
X_train, y_train = utils.load_dataset(os.path.join('data', 'creditdata', 'creditdata_train_v2.csv'), drop_columns=drop_cols)
X_test, y_test = utils.load_dataset(os.path.join('data', 'creditdata', 'creditdata_test_v2.csv'), drop_columns=drop_cols)

In [7]:
# Base default rates
print(f"Number of features: {len(X_train.columns)}")
print(f"Train set length: {len(y_train)}, default rate: {round(1 - np.mean(y_train), 4)}")
print(f"Test set length: {len(y_test)}, default rate: {round(1 - np.mean(y_test), 4)}")
print(X_train.columns)

Number of features: 23
Train set length: 22500, default rate: 0.2212
Test set length: 7500, default rate: 0.2212
Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')


## Train Model

In [8]:
# Use best parameters from a model selection and threshold tuning process
best_regularizer = 1e-1
best_th = 0.43
model = utils.train_log_reg_model(X_train, y_train, seed=0, C=best_regularizer, upsample=True, verbose=True)

upsampling...
scaling...
fitting...
C: 0.1
chaining pipeline...
done.


## Test Performance
Here we quantify the model's performance.

In [9]:
# Get prediction probabilities and predictions at risk threshold
def prob_of_good(model, X):
    return model.predict_proba(X)[:, 1] # prob of resolve (good)

test_probs = prob_of_good(model, X_test)
assert (max(test_probs) < 1) and (max(test_probs) > 0)

test_preds = np.where(test_probs > best_th, 1, 0)

In [10]:
# Simple computation
test_bal_acc = metrics.balanced_accuracy_score(y_test, test_preds)
print(f"Balanced accuracy on test set {round(test_bal_acc, 5)} at threshold {round(best_th, 5)}")

Balanced accuracy on test set 0.68451 at threshold 0.43


In [11]:
def plot_roc(model, X, y):
    figure = plt.figure(figsize=(5,5))
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Random Chance', alpha=.8)
    metrics.plot_roc_curve(model, X, y, name='model', alpha=0.3, lw=2, ax=plt.gca())
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=13)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.show()

plot_roc(model, X_test, y_test)

In [12]:
# Bootstrap uncertainty analysis

# Metrics based on predictions
prediction_metrics = {'True Positive Rate (i.e. sensitivity, or recall)': metrics.recall_score,
                      'True Negative Rate (i.e. specificity)': lambda x, y: metrics.recall_score(x, y, pos_label=0),
                      'Balanced Accuracy': metrics.balanced_accuracy_score,
                      'Positive Predictive Value (precision)': metrics.precision_score}

# Metrics based on probabilities
probability_metrics = {'Area Under ROC': metrics.roc_auc_score}

for name, metric_func in prediction_metrics.items():
    print(name, ":", utils.format_uncertainty(*utils.bootstrap_conf_int(y_test, test_preds, metric_func, k=25)))

for name, metric_func in probability_metrics.items():
    print(name, ":", utils.format_uncertainty(*utils.bootstrap_conf_int(y_test, test_probs, metric_func, k=25)))



True Positive Rate (i.e. sensitivity, or recall) : 0.86126 +/- 0.00920
True Negative Rate (i.e. specificity) : 0.50537 +/- 0.02193
Balanced Accuracy : 0.68499 +/- 0.01074
Positive Predictive Value (precision) : 0.85935 +/- 0.00746
Area Under ROC : 0.72393 +/- 0.01278


In [13]:
# Calibration curve
def plot_calibration(bin_true_prob, bin_pred_prob):
    plt.figure(figsize=(7, 7))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="perfectly calibrated")
    ax1.plot(bin_pred_prob, bin_true_prob, "s-",
             label="model")

    ax2.hist([test_probs[y_test == 1], test_probs[y_test == 0]], label=["resolve", "default"],
              histtype='bar', stacked=True)

    ax1.set_ylabel("Fraction of loans that resolve", fontsize=14)
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right", fontsize=12)
    ax1.set_title('Model Calibration (reliability curve)', fontsize=16)

    ax2.set_xlabel("Model Output Probabilities (binned)", fontsize=14)
    ax2.set_ylabel("Count", fontsize=14)
    ax2.legend(loc="upper left", ncol=2, fontsize=12)

    plt.tight_layout()
    plt.show()

bin_true_prob, bin_pred_prob = calibration_curve(y_test, test_probs, n_bins=10)
plot_calibration(bin_true_prob, bin_pred_prob)

## Fairness
Here we compute some fairness metrics with respect to gender.

In [14]:
# Create a gender identifying mask
gender_mask = (X_test["SEX"] == 1)  # True: Male, False: Female 
print('Percent Male:', round(np.mean(gender_mask), 5), 'Percent Female:', round(np.mean(~gender_mask), 5))

Percent Male: 0.40267 Percent Female: 0.59733


In [15]:
# Run the analysis
gender_analysis = utils.FairnessAnalysis(y_test, test_probs, gender_mask)
gender_metrics = gender_analysis.compute(best_th)
for attr, name in utils.FairnessAnalysis.metric_names.items():
    print(name, ":", round(getattr(gender_metrics, attr), 5))

Equal Opportunity : -0.05668
False Positive Rate Balance : -0.02355
Average Odds : -0.04011
Demographic Parity : -0.06753
Positive Predictive Parity : -0.03778
Balanced Accuracy : 0.6846


In [16]:
# Bootstrap Uncertainty
bs_metrics = []
np.random.seed(0)
for i in range(25):
    idx = np.random.choice(len(y_test), len(y_test), replace=True)
    bs_metrics.append(utils.FairnessAnalysis(y_test[idx], test_probs[idx], gender_mask[idx]).compute(best_th))

bs_metrics = np.array(bs_metrics)

In [17]:
for i, attr in enumerate(gender_metrics._fields):
    print(utils.FairnessAnalysis.metric_names[attr], ":", utils.format_uncertainty(bs_metrics[:, i].mean(), 2 * bs_metrics[:, i].std()))

Equal Opportunity : -0.05565 +/- 0.01901
False Positive Rate Balance : -0.02197 +/- 0.04882
Average Odds : -0.03881 +/- 0.02327
Demographic Parity : -0.06627 +/- 0.01734
Positive Predictive Parity : -0.03838 +/- 0.01963
Balanced Accuracy : 0.68368 +/- 0.01384


## Performance-Fairness Tradeoffs
Here we explore fairness-performance tradeoffs stemming from our choice of lending threshold.

In [18]:
# Split Threshold Sweep
th_a = np.linspace(0.3, 0.7, 500)
th_b = np.linspace(0.3, 0.7, 500)
grid_th_a, grid_th_b = np.meshgrid(th_a, th_b, sparse=True)
gender_metrics_split_sweep = gender_analysis.compute(grid_th_a, grid_th_b)

In [19]:
# Find best split threshold
bal_acc_grid = gender_metrics_split_sweep.bal_acc
idx = np.unravel_index(bal_acc_grid.argmax(), bal_acc_grid.shape)
best_th_a, best_th_b = th_a[idx[1]], th_b[idx[0]]
print(f"Best Balanced Accuracy (split): {bal_acc_grid.max():.5f} with Male TH: {best_th_a:.3f}, Female TH: {best_th_b:.3f}")

Best Balanced Accuracy (split): 0.68784 with Male TH: 0.431, Female TH: 0.422


In [20]:
# Find bal accuracy when fairness constrained
constrained_bal_acc = np.copy(bal_acc_grid)
constrained_bal_acc[np.where(gender_metrics_split_sweep.equal_opp <= -0.000)] = 0
idx = np.unravel_index(constrained_bal_acc.argmax(), constrained_bal_acc.shape)
best_con_th_a, best_con_th_b = th_a[idx[1]], th_b[idx[0]]
print(f"Best Fairness-Constrained Balanced Accuracy: {constrained_bal_acc.max():.5f} with Male TH: {best_con_th_a:.3f}, Female TH: {best_con_th_b:.3f}")

Best Fairness-Constrained Balanced Accuracy: 0.68418 with Male TH: 0.412, Female TH: 0.443


In [21]:
# Plot
plt.figure(figsize=(9,8))
plt.title('Fairness vs. Performance Tradeoffs', fontsize=18)
plt.xlabel('Lending Threshold Men', fontsize=16)
plt.ylabel('Lending Threshold Women', fontsize=16)
plt.xlim(np.min(th_a), np.max(th_a))
plt.ylim(np.min(th_b), np.max(th_b))

bal_acc_lns = plt.contourf(th_a, th_b, gender_metrics_split_sweep.bal_acc, levels=20)

eo_lns = plt.contour(th_a, th_b, gender_metrics_split_sweep.equal_opp, colors='white', levels=10)
eo_lns.collections[-1].set_label('equal opportunity')

cbar = plt.colorbar(bal_acc_lns)
cbar.set_label('Model Performance (balanced accuracy)', fontsize=14)
plt.clabel(eo_lns, inline=1,fmt='%1.2f', fontsize=14)

# Mark maximums
# plt.plot([0, 1], [0, 1], c='gray', ls=':', label='single threshold')
plt.scatter(best_th_a, best_th_b, c='b', marker='d', s=100, label= 'max bal acc', zorder=2)
plt.scatter(best_th, best_th, c='r', marker='x', s=100, label= 'single TH bal acc', zorder=2)
plt.scatter(best_con_th_a, best_con_th_b, c='purple', marker='*', s=100, label= 'equal opp bal acc', zorder=2)
lgnd = plt.legend(framealpha=0.3, facecolor='black', fontsize=12, loc='upper right')
for text in lgnd.get_texts():
    text.set_color("white")
plt.show()

#### Fairness-Performance Tradeoffs (split threshold)
Once the analysis has been run, we can visualize the fairness-performance tradeoffs of operating the model at various lending risk threshold settings. Operating the model at a higher lending risk threshold equates to a lower loan approval rate. The x-axis shows a range of possible lending risk thresholds for men, 
while the y-axis shows a range of possible lending risk thresholds for women. 

The heatmap indicates the model's expected performance (balanced accuracy) when operated at each pair of risk thresholds. Balanced accuracy is the average of the true positive rate (TPR), (i.e. sensitivity, or recall), and the true negative rate (TNR) (i.e. specificity). In our AIDA credit approval system, we use (a possibly weighted) balanced accuracy as a stand-in for the expected gross profit (this is further explored in the report).

The white contour lines indicate the equal opportunity group fairness metric with respect to gender. Equal opportunity measures the difference in the true positive rates between two groups of individuals, in this case men and women. It is computed as `TPR_men - TPR_women`, thus it is optimal when equal to zero (0). The true positive rate corresponds to the probability that an applicant who would hypothetically repay their loan is accepted by the model.

We plot three points of interest. 
The blue diamond maximizes the unconstrained model performance.
The red X maximizes model performance while keeping the *same* lending risk threshold for both men and women.
The purple star maximizes the model performance while ensuring optimal gender fairness as measured via equal opportunity.

## Personal Attributes
Here we consider how we might justify the inclusion of personal attributes

In [22]:
personal_attrs = ['SEX', 'EDUCATION', 'MARRIAGE', 'AGE']

In [23]:
# Leave one out analysis
loo_metrics = []
for i, attr in enumerate(personal_attrs):
    print('\nTraining model without:', attr)
    model_loo = utils.train_log_reg_model(X_train.drop(columns=[attr]), y_train, seed=0, C=best_regularizer, verbose=False)
    
    # Predict and compute fairness Metrics
    loo_test_probs = prob_of_good(model_loo, X_test.drop(columns=[attr]))
    loo_analysis = utils.FairnessAnalysis(y_test, loo_test_probs, gender_mask)
    loo_metrics.append(loo_analysis.compute(best_th))
    
    # Display results as they arrive
    for field, name in utils.FairnessAnalysis.metric_names.items():
        print(name, ":", round(getattr(loo_metrics[i], field), 5))


Training model without: SEX
Equal Opportunity : -0.03221
False Positive Rate Balance : 0.00774
Average Odds : -0.01223
Demographic Parity : -0.04141
Positive Predictive Parity : -0.04228
Balanced Accuracy : 0.68271

Training model without: EDUCATION
Equal Opportunity : -0.0452
False Positive Rate Balance : -0.01531
Average Odds : -0.03026
Demographic Parity : -0.05681
Positive Predictive Parity : -0.03847
Balanced Accuracy : 0.68432

Training model without: MARRIAGE
Equal Opportunity : -0.05304
False Positive Rate Balance : -0.00349
Average Odds : -0.02826
Demographic Parity : -0.05981
Positive Predictive Parity : -0.043
Balanced Accuracy : 0.68117

Training model without: AGE
Equal Opportunity : -0.06084
False Positive Rate Balance : -0.02414
Average Odds : -0.04249
Demographic Parity : -0.07111
Positive Predictive Parity : -0.03792
Balanced Accuracy : 0.68702


In [24]:
# # Transform into percent differences
# bal_acc_deltas = [100 * (loo.bal_acc - gender_metrics.bal_acc) / gender_metrics.bal_acc for loo in loo_metrics]
# equal_opp_deltas = [100 * (loo.equal_opp - gender_metrics.equal_opp) / np.abs(gender_metrics.equal_opp) for loo in loo_metrics]

# Keep as absolutes
bal_acc_deltas = [loo.bal_acc - gender_metrics.bal_acc for loo in loo_metrics]
equal_opp_deltas = [loo.equal_opp - gender_metrics.equal_opp for loo in loo_metrics]

In [25]:
# Plot
plt.figure(figsize=(8,6))
plt.title('Impact of Personal Attributes', fontsize=18)
x = np.arange(len(personal_attrs))  # the label locations
width = 0.35  # the width of the bars
rects1 = plt.bar(x - width/2, bal_acc_deltas, width, label='Balanced Acc.')
rects2 = plt.bar(x + width/2, equal_opp_deltas, width, label='Eq. Opp. (gender)')
plt.axhline(0, c='k', ls='-', lw='1')
plt.axhline(-gender_metrics.equal_opp, c='k', ls=':', label='Neutral Eq. Opp.') # show neutrality
plt.xticks(x, personal_attrs, rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Effect of Removal (removed - baseline)', fontsize=16)
plt.legend(fontsize=12)
plt.show()

#### Justifying the use of Personal Attributes
After running a "leave-one-out" feature removal analysis, we can assess the approximate impact of personal attributes on both fairness and model performance. We plot the impact of removing each personal attribute on the model's performance (balanced accuracy) and the equal opportunity group fairness metric with respect to gender. We want balanced accuracy to be as high a possible, while ideally equal opportunity would be at neutrality. 

**Tradeoffs to be further examined**: Attributes for which removal negatively affect model performance but positively affect the fairness metric(s) of interest (or vice-versa). In this example, SEX, EDUCATION, MARRIAGE, and AGE. 

**Evidence for inclusion**: Attributes for which removal negatively affect both model performance and the fairness metric(s) of interest. In this example, none. 

**Evidence for exclusion**: Attributes for which removal positively affect both model performance and the fairness metric(s) of interest. In this example, none.